# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [6]:
import math

In [7]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)
suma = 0.0

for sqrt_value in generator:
    suma += sqrt_value
    print(f"{suma:.3f}")


1.000
2.414
4.146
6.146
8.382


In [8]:
round(sum(square_root_generator(5)), 3)

8.382

In [9]:
round(max(square_root_generator(13)), 3)

3.606

In [10]:
generator_2 = square_root_generator(13)
for value in generator_2:
    print(f"{value:.3f}")

1.000
1.414
1.732
2.000
2.236
2.449
2.646
2.828
3.000
3.162
3.317
3.464
3.606


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [11]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [12]:
!pip install duckdb

In [13]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

con.execute("CREATE TABLE people (ID INTEGER, Name VARCHAR, Age INTEGER, City VARCHAR, Occupation VARCHAR)")

In [14]:
con.sql("DESCRIBE TABLE people")

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ID          │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ Name        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Age         │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ City        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Occupation  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [15]:
def insert_generator(generator):
    for person in generator:
        try:
            person["Occupation"]
        except KeyError:
            person["Occupation"] = None
        con.execute("INSERT INTO people VALUES (?, ?, ?, ?, ?)", list(person.values()))

In [16]:
insert_generator(people_1())

In [17]:
con.sql("SELECT SUM(Age) FROM people")

┌──────────┐
│ sum(Age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

In [18]:
insert_generator(people_2())

In [19]:
con.sql("SELECT SUM(Age) FROM people")

┌──────────┐
│ sum(Age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [20]:
con.execute(
    """
    CREATE TABLE people_pk (
        ID INTEGER PRIMARY KEY,
        Name VARCHAR,
        Age INTEGER,
        City VARCHAR,
        Occupation VARCHAR
    )
    """
)

In [21]:
con.sql("DESCRIBE people_pk")

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ ID          │ INTEGER     │ NO      │ PRI     │ NULL    │  NULL │
│ Name        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Age         │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ City        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Occupation  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴───────┘

In [22]:
def upsert_generator(generator):
    for person in generator:
        try:
            person["Occupation"]
        except KeyError:
            person["Occupation"] = None
        con.execute(
            """
            INSERT INTO people_pk
            VALUES (?, ?, ?, ?, ?)
            ON CONFLICT (ID)
            DO UPDATE SET
                Name = EXCLUDED.Name,
                Age = EXCLUDED.Age,
                City = EXCLUDED.City,
                Occupation = EXCLUDED.Occupation
            """,
            list(person.values())
        )

In [23]:
upsert_generator(people_1())

In [24]:
con.sql("select * from people_pk")

┌───────┬──────────┬───────┬─────────┬────────────┐
│  ID   │   Name   │  Age  │  City   │ Occupation │
│ int32 │ varchar  │ int32 │ varchar │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ NULL       │
└───────┴──────────┴───────┴─────────┴────────────┘

In [25]:
upsert_generator(people_2())

In [26]:
con.sql("select * from people_pk")

┌───────┬──────────┬───────┬─────────┬────────────┐
│  ID   │   Name   │  Age  │  City   │ Occupation │
│ int32 │ varchar  │ int32 │ varchar │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ Job_4      │
│     5 │ Person_5 │    35 │ City_B  │ Job_5      │
│     6 │ Person_6 │    36 │ City_B  │ Job_6      │
│     7 │ Person_7 │    37 │ City_B  │ Job_7      │
│     8 │ Person_8 │    38 │ City_B  │ Job_8      │
└───────┴──────────┴───────┴─────────┴────────────┘

In [27]:
con.sql("select sum(Age) as total_age from people_pk")

┌───────────┐
│ total_age │
│  int128   │
├───────────┤
│       266 │
└───────────┘

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX

# Playing with DuckDB

In [28]:
base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022"

In [45]:
sql_query = "SELECT * FROM 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet' LIMIT 100"

In [38]:
con.sql("INSTALL https;")

In [39]:
con.sql("LOAD https;")

In [46]:
con.sql(sql_query)

┌──────────┬──────────────────────┬──────────────────────┬───┬──────────────┬───────────┬──────────────────────┐
│ VendorID │ lpep_pickup_datetime │ lpep_dropoff_datet…  │ … │ payment_type │ trip_type │ congestion_surcharge │
│  int64   │      timestamp       │      timestamp       │   │    double    │  double   │        double        │
├──────────┼──────────────────────┼──────────────────────┼───┼──────────────┼───────────┼──────────────────────┤
│        2 │ 2022-01-01 00:14:21  │ 2022-01-01 00:15:33  │ … │          2.0 │       1.0 │                  0.0 │
│        1 │ 2022-01-01 00:20:55  │ 2022-01-01 00:29:38  │ … │          2.0 │       1.0 │                  0.0 │
│        1 │ 2022-01-01 00:57:02  │ 2022-01-01 01:13:14  │ … │          1.0 │       1.0 │                 2.75 │
│        2 │ 2022-01-01 00:07:42  │ 2022-01-01 00:15:57  │ … │          2.0 │       1.0 │                  0.0 │
│        2 │ 2022-01-01 00:07:50  │ 2022-01-01 00:28:52  │ … │          1.0 │       1.0 │       

In [47]:
con.sql("DESCRIBE FROM 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet'")

┌───────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│      column_name      │ column_type │  null   │   key   │ default │  extra  │
│        varchar        │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ VendorID              │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ lpep_pickup_datetime  │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ lpep_dropoff_datetime │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ store_and_fwd_flag    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ RatecodeID            │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ PULocationID          │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ DOLocationID          │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ passenger_count       │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ trip_distance         │ DOUBLE      │ 

In [53]:
con.sql("SELECT VendorID, RatecodeID, sum(tip_amount) as tip FROM 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet' group by all")

AttributeError: This relation does not contain a column by the name of 'polars'

In [55]:
some_data = [{"id": 1, "value": 10}, {"id": 2, "value": 20}, {"id": 3, "value": 30}]

In [58]:
import pandas as pd

In [59]:
internal_df = pd.DataFrame(some_data)

In [60]:
internal_df

,id,value
0,1,10
1,2,20
2,3,30


In [80]:
%time con.sql("EXPLAIN ANALYZE SELECT * from internal_df CROSS JOIN (SELECT VendorID, RatecodeID, sum(tip_amount) as tip FROM 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet' group by all)")

CPU times: user 231 ms, sys: 606 µs, total: 232 ms
Wall time: 535 ms


┌───────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  explain_key  │                                            explain_value                                             │
│    varchar    │                                               varchar                                                │
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ analyzed_plan │ ┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││    Query Prof…  │
└───────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────────┘
